In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import datetime as dt
import requests
import yfinance as yf
from time import sleep
import re
import bisect

import bs4
from bs4 import BeautifulSoup

sns.set_style('darkgrid')

In [4]:
from api_keys import sec_email

In [5]:
headers = {"User-Agent": sec_email} # Your email goes here

def NPORT_Filings_from_CIK(cik, headers=headers):
    headers = headers
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    sleep(2)
    filings = requests.get(url, headers=headers).json()
    filings_df = pd.DataFrame(filings["filings"]["recent"])
    nport_filings_df = filings_df[filings_df["form"] == "NPORT-P"]
    nport_filings_df.loc[:,'filingDate'] = pd.to_datetime(nport_filings_df['filingDate'])
    nport_filings_df.loc[:,'reportDate'] = pd.to_datetime(nport_filings_df['reportDate'])
    return nport_filings_df

def gen_company_name_and_cik_list(headers=headers):
    headers = headers
    url = f"https://www.sec.gov/Archives/edgar/cik-lookup-data.txt"
    sleep(2)
    response = requests.get(url, headers=headers)
    #filings_df = pd.DataFrame(filings)
    return response.text.split('\n')

cik_list = gen_company_name_and_cik_list()

def holdings_from_NPORT(accessionNumber,primaryDocument,reportDate,headers=headers):

    sleep(2)

    url = f"https://www.sec.gov/Archives/edgar/data/1064641/{accessionNumber}/{primaryDocument}"
    
    response = requests.get(url, headers=headers)

    assert response.status_code == 200

    soup = BeautifulSoup(response.text, 'html.parser')

    stocks_source = soup.findAll("td",string='a. Name of issuer (if any). \n\t\t\t\t')
    CUSIPs_source = soup.findAll("td",string='d. CUSIP (if any).\n\t\t\t\t')
    weights_source = soup.findAll('td',string='Percentage value compared to net assets of the Fund.\n\t\t\t')

    stocks = [stock.parent.find('div').contents[0] for stock in stocks_source]
    CUSIPs = [CUSIP.parent.find('div').contents[0] for CUSIP in CUSIPs_source]
    weights = [weight.parent.find('div').contents[0] for weight in weights_source]
    CIKs = []

    for stock in stocks:
        index = bisect.bisect_left(cik_list, stock.upper())
        CIKs.append(cik_list[index].split(":")[1])

    holdings = pd.DataFrame({'Stock' : stocks,'CIK' : CIKs,f'Weighting in quarter starting {reportDate}' : weights})
    
    elem = soup.findAll('td',string="Series ID")[0]
    
    seriesID = elem.parent.parent.div.contents[0]
    
    return seriesID, holdings

In [6]:
cik_list

['!J INC:0001438823:',
 '#1 A LIFESAFER HOLDINGS, INC.:0001509607:',
 '#1 ARIZONA DISCOUNT PROPERTIES LLC:0001457512:',
 '#1 PAINTBALL CORP:0001433777:',
 '$ LLC:0001427189:',
 '$AVY, INC.:0001655250:',
 '& S MEDIA GROUP LLC:0001447162:',
 '&OPEN GIFTS LTD:0002010293:',
 '&PARTNERS:0000107136:',
 '&TV COMMUNICATIONS INC.:0001479357:',
 '&VEST DOMESTIC FUND II KPIV, L.P.:0001802417:',
 '&VEST DOMESTIC FUND II LP:0001800903:',
 '&VEST OFFSHORE FUND II L.P.:0001800902:',
 "'LAINE'S BAKE SHOP LLC, SERIES OF BG CONSORTIUM LLC:0001732207:",
 "'MERICAN MULE, INC.:0001833137:",
 "'MKTG, INC.':0000886475:",
 "'OHANA LABS INC.:0001703629:",
 "'PARAGON VMS USA CORP:0001786139:",
 "'T HART CEES:0001983327:",
 '(OURCROWD INVESTMENT IN MST) L.P.:0001599496:',
 '(Y.Z) QUEENCO LTD.:0001623088:',
 '.406 VENTURES I GP, L.P.:0001854758:',
 '.406 VENTURES I GP, LLC:0001854757:',
 '.406 VENTURES MANAGEMENT, L.P.:0001766976:',
 '.CLUB DOMAINS, LLC:0001577787:',
 '001 - SERIES OF IPOSHARKS VENTURE MASTER FUN

In [7]:
filings = NPORT_Filings_from_CIK('0001064641')

SPDR_holdings = {}

for i in range(len(filings)):
    accessionNumber = filings["accessionNumber"].iloc[i].replace("-","")
    primaryDocument = filings["primaryDocument"].iloc[i]
    reportDate = filings["reportDate"].iloc[i]

    seriesID, holdings = holdings_from_NPORT(accessionNumber,primaryDocument,reportDate,headers=headers)

    if seriesID in SPDR_holdings:
        SPDR_holdings[seriesID].merge(holdings,how='outer')
    else:
        SPDR_holdings[seriesID] = holdings

In [32]:
SPDR_holdings

{'S000006414':                                      Stock         CIK  \
 0                                 FMC Corp  0000037785   
 1                           Albemarle Corp  0000915913   
 2                                  Dow Inc  0001751788   
 3                                Ball Corp  0000009389   
 4           Air Products and Chemicals Inc  0001838153   
 5                                Amcor PLC  0001748790   
 6                            Mosaic Co/The  0001285785   
 7                            Celanese Corp  0000018443   
 8                       PPG Industries Inc  0000079879   
 9                               Nucor Corp  0000073309   
 10                             Corteva Inc  0001755672   
 11                     Vulcan Materials Co  0000103973   
 12                     Eastman Chemical Co  0000915377   
 13               Packaging Corp of America  0000075677   
 14                            Newmont Corp  0001164727   
 15                   DuPont de Nemours In

In [75]:
type(filings)

pandas.core.frame.DataFrame

In [82]:
import pickle
import json
# for some reason cannot save, so added recursion limit
import sys
sys.setrecursionlimit(10000)

In [40]:
i = 0
for key in SPDR_holdings.keys():
    print(SPDR_holdings[key])
    print(type(SPDR_holdings[key]), "   NEW ELEMENT \n\n\n\n\n")
    i+=1

print("Num dicts: ",i)

                                     Stock         CIK  \
0                                 FMC Corp  0000037785   
1                           Albemarle Corp  0000915913   
2                                  Dow Inc  0001751788   
3                                Ball Corp  0000009389   
4           Air Products and Chemicals Inc  0001838153   
5                                Amcor PLC  0001748790   
6                            Mosaic Co/The  0001285785   
7                            Celanese Corp  0000018443   
8                       PPG Industries Inc  0000079879   
9                               Nucor Corp  0000073309   
10                             Corteva Inc  0001755672   
11                     Vulcan Materials Co  0000103973   
12                     Eastman Chemical Co  0000915377   
13               Packaging Corp of America  0000075677   
14                            Newmont Corp  0001164727   
15                   DuPont de Nemours Inc  0001666700   
16            

In [44]:
for key in SPDR_holdings.keys():
    print(key)
    print(SPDR_holdings[key].keys())
    #SPDR_holdings[key].to_pickle('data/'+key+'.txt')
        

S000006414
Index(['Stock', 'CIK', 'Weighting in quarter starting 2023-12-31 00:00:00'], dtype='object')
S000006409
Index(['Stock', 'CIK', 'Weighting in quarter starting 2023-12-31 00:00:00'], dtype='object')
S000051152
Index(['Stock', 'CIK', 'Weighting in quarter starting 2023-12-31 00:00:00'], dtype='object')
S000006416
Index(['Stock', 'CIK', 'Weighting in quarter starting 2023-12-31 00:00:00'], dtype='object')
S000006411
Index(['Stock', 'CIK', 'Weighting in quarter starting 2023-12-31 00:00:00'], dtype='object')
S000006410
Index(['Stock', 'CIK', 'Weighting in quarter starting 2023-12-31 00:00:00'], dtype='object')
S000062095
Index(['Stock', 'CIK', 'Weighting in quarter starting 2023-12-31 00:00:00'], dtype='object')
S000006413
Index(['Stock', 'CIK', 'Weighting in quarter starting 2023-12-31 00:00:00'], dtype='object')
S000006412
Index(['Stock', 'CIK', 'Weighting in quarter starting 2023-12-31 00:00:00'], dtype='object')
S000006408
Index(['Stock', 'CIK', 'Weighting in quarter starting

In [46]:
#XXX This was the issue I was running into when trying to save the dicts. Leaving for posterity.
# It turns out a few of the columns were saved as Navigable Strings from BeautifulSoup. 
# These cause the recursion depth issue when saving for some reason. Code from below can be used for conversion
SPDR_holdings['S000006414'].to_pickle('test.txt')

RecursionError: maximum recursion depth exceeded while pickling an object

In [47]:
testdf = SPDR_holdings['S000006414'].copy()

In [48]:
testdf.keys()

Index(['Stock', 'CIK', 'Weighting in quarter starting 2023-12-31 00:00:00'], dtype='object')

In [63]:
testdf.rename(columns={"Stock":"Stock","CIK":"CIK","Weighting in quarter starting 2023-12-31 00:00:00":"20231231"},inplace=True)

In [161]:
def save_all(holdings_dict):
    for key in holdings_dict:
        date_str = holdings_dict[key].keys()[2].split()[4].replace('-','')
        df_temp = holdings_dict[key].rename(columns={"Stock":"Stock","CIK":"CIK",holdings_dict[key].keys()[2]:'Weight'})
        df_temp['Stock'] = df_temp['Stock'].apply(str)
        df_temp['Weight'] = df_temp['Weight'].apply(str)
        df_temp.to_pickle('data/'+key+'_'+date_str+'.pkl')
        #print(type(df_temp['20231231'][0]))
        #df_temp['20231231'] = str(df_temp['20231231'])
        #print(df_temp)

In [162]:
save_all(SPDR_holdings)

In [103]:
print(SPDR_holdings)

{'S000006414':                                      Stock         CIK  \
0                                 FMC Corp  0000037785   
1                           Albemarle Corp  0000915913   
2                                  Dow Inc  0001751788   
3                                Ball Corp  0000009389   
4           Air Products and Chemicals Inc  0001838153   
5                                Amcor PLC  0001748790   
6                            Mosaic Co/The  0001285785   
7                            Celanese Corp  0000018443   
8                       PPG Industries Inc  0000079879   
9                               Nucor Corp  0000073309   
10                             Corteva Inc  0001755672   
11                     Vulcan Materials Co  0000103973   
12                     Eastman Chemical Co  0000915377   
13               Packaging Corp of America  0000075677   
14                            Newmont Corp  0001164727   
15                   DuPont de Nemours Inc  0001666700   

In [134]:
df_temp = SPDR_holdings['S000006414'].rename(columns={"Stock":"Stock","CIK":"CIK","Weighting in quarter starting 2023-12-31 00:00:00":"20231231"})
df_temp['Stock'] = df_temp['Stock'].apply(str)
df_temp['20231231'] = df_temp['20231231'].apply(str)
print(type(df_temp['20231231'][0]))
#df_temp['20231231'] = str(df_temp['20231231'])
print(df_temp)

<class 'str'>
                                     Stock         CIK        20231231
0                                 FMC Corp  0000037785  0.820804808039
1                           Albemarle Corp  0000915913  1.769259457977
2                                  Dow Inc  0001751788  4.013604363036
3                                Ball Corp  0000009389  1.892434479561
4           Air Products and Chemicals Inc  0001838153  6.294291317730
5                                Amcor PLC  0001748790  1.453853774256
6                            Mosaic Co/The  0001285785  1.218513921457
7                            Celanese Corp  0000018443  1.623646490888
8                       PPG Industries Inc  0000079879  3.679573433746
9                               Nucor Corp  0000073309  4.464471934443
10                             Corteva Inc  0001755672  3.523813744966
11                     Vulcan Materials Co  0000103973  3.147400934344
12                     Eastman Chemical Co  0000915377  1.11120

In [4]:
df_new = pd.read_pickle('data/S000062095_20231231.pkl')

In [140]:
df_new

,Stock,CIK,20231231
0,Warner Bros Discovery Inc,0001437107,3.394040997044
1,News Corp,0001039204,1.255737799736
2,Omnicom Group Inc,0000029989,2.301362867976
3,Comcast Corp,0000022301,4.522192690129
4,Fox Corp,0001754301,0.985845959811
5,Alphabet Inc,0001652044,12.39992502325
6,Paramount Global,0000813828,0.959015802228
7,Take-Two Interactive Software Inc,0001792627,3.421393671219
8,Alphabet Inc,0001652044,10.52877776326
9,Walt Disney Co/The,0000926480,4.223361640333
